In [1]:
dataset = r"C:\Users\Arun\Downloads\dataset\dataset"

In [2]:
from torchvision import transforms

augmentation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2),
    transforms.Normalize(mean=[0.4784, 0.4774, 0.4246],std=[0.2180, 0.2144, 0.2105])
])

In [3]:
from torchvision.datasets import ImageFolder

data = ImageFolder(root = dataset,transform = augmentation)

In [4]:
train_size = int(0.8* len(data))
test_size = len(data)-train_size

In [5]:
from torch.utils.data import random_split,DataLoader
training_data,validation_data = random_split(data,[train_size,test_size])

In [6]:
training_data=DataLoader(training_data,batch_size=1024,shuffle=True,num_workers=3)
test_data=DataLoader(validation_data,batch_size=128,shuffle=True,num_workers=3)

In [7]:
# mean = 0.0
# std = 0.0
# nb_samples = 0
# for images, _ in training_data:
#     batch_samples = images.size(0)
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)
#     nb_samples+=batch_samples
# mean /= nb_samples
# std /= nb_samples
#
# print(mean, std)

In [8]:
import torch.nn as nn

In [9]:
class animal_cnn(nn.Module):
    def __init__(self,num_classes):
        super(animal_cnn,self).__init__()
        self.features=nn.Sequential(
            nn.Conv2d(3,32,3,padding=1),nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,128,3,padding=1),nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.AdaptiveAvgPool2d((8,8)),
            nn.Flatten(),
            nn.Linear(128*8*8,256),
            nn.Dropout(0.35),
            nn.Linear(256,num_classes)
            )
    def forward(self,x):
        return self.features(x)

In [10]:
import torch

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [13]:
model = animal_cnn(151).to(device)
model_loss = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(),lr=0.0001,weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optim,mode="min",patience=5)

In [14]:
class earlystopping():
    def __init__(self,min_delta=0.01,patience=5):
        self.patience = patience
        self.min_delta = min_delta
        self.early_stoppiing = False
        self.best_loss = float('inf')
        self.counter = 0
    def __call__(self,val_loss):
        if val_loss < self.best_loss - self.min_delta :
            self.counter=0
            self.best_loss = val_loss
        else:
            self.counter +=1
            if self.counter>=self.patience:
                self.early_stoppiing = True

In [15]:
from torch.amp import autocast,GradScaler

In [ ]:
scaler = GradScaler('cuda')

epochs = 50
training_loss = []
testing_loss = []
accuracy = []
test_accuracy = []

earlystopping_call = earlystopping()

for epoch in range(epochs):
    model.train()
    running_loss, mcorrect, mtotal = 0, 0, 0

    for images, labels in training_data:
        images, labels = images.to(device), labels.to(device)
        optim.zero_grad()

        with autocast('cuda'):
            outputs = model(images)
            loss = model_loss(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()

        running_loss += loss.item()
        _, pred = torch.max(outputs, 1)
        mcorrect += (pred == labels).sum().item()
        mtotal += labels.size(0)

    train_loss = running_loss / len(training_data)
    train_acc = 100 * mcorrect / mtotal
    training_loss.append(train_loss)
    accuracy.append(train_acc)

    model.eval()
    vrunning_loss, vcorrect, vtotal = 0, 0, 0
    with torch.no_grad(), autocast('cuda'):
        for images, labels in test_data:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss = model_loss(outputs, labels)

            vrunning_loss += test_loss.item()
            _, pred = torch.max(outputs, 1)
            vcorrect += (pred == labels).sum().item()
            vtotal += labels.size(0)

    val_loss = vrunning_loss / len(test_data)
    val_acc = 100 * vcorrect / vtotal
    testing_loss.append(val_loss)
    test_accuracy.append(val_acc)

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}: Train Acc {train_acc:.2f}% | Train Loss {train_loss:.4f} | "
          f"Test Acc {val_acc:.2f}% | Test Loss {val_loss:.4f}")

    # earlystopping_call(val_loss)
    # if earlystopping_call.early_stopping:
    #     print("Early stopping triggered.")
    #     break
